# Analytic & Rosenbrock results

Includes the comparison to nessai.

Michael J. Williams 2023

In [ ]:
import os
import sys

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns
from scipy import stats
import statsmodels.api as sm

basedir = "../"

sys.path.append(basedir)

from utils import configure_plotting, load_all_results
    
configure_plotting(basedir)
default_figsize = plt.rcParams["figure.figsize"]

In [ ]:
path = './outdir_rerun/'
gaussian_path = os.path.join(path, f'ins_gaussian_*d', '')
gmm_path = os.path.join(path, f'ins_gmm_paper_*d', '')

In [ ]:
gaussian_results = load_all_results(gaussian_path)

In [ ]:
gmm_results = load_all_results(gmm_path)

In [ ]:
possible_dims = np.arange(2, 34, 2)
truth = {n: -n * np.log(20) for n in possible_dims}

In [ ]:
figsize = ((10 / 13) * default_figsize[0], default_figsize[1] * (1.8 / 5))

results = [gaussian_results, gmm_results]
hspace = 0.1
# Ratio of height to width scaled by the number of subplots in each dimension
wspace = hspace * (figsize[1] / figsize[0]) * (2 / 1)

fig, axs = plt.subplots(
    1, len(results), sharey=True, figsize=figsize,
    gridspec_kw={"hspace": hspace, "wspace": wspace}
)

offset_width = 0.15
offsets = offset_width * (np.arange(len(results)) - (len(results) - 1)/ 2)

for i, res in enumerate(results):
    dims = np.array(list(res.keys()))
    log_z = np.array([d['log_evidence'].mean() - truth[k] for k, d in res.items()])
    log_z_err = np.array([d['log_evidence_error'].mean() for d in res.values()])
    final_log_z = np.array([d['final_log_evidence'].mean() - truth[k] for k, d in res.items()])
    final_log_z_err = np.array([d['final_log_evidence_error'].mean() for d in res.values()])

    print(f"Results for model {i}")
    print("log Z: ", log_z)
    print("final_log Z: ", final_log_z)
    print(f"Average bias: {np.abs(100 * log_z / dims * np.log(20))} %")
    
    axs[i].errorbar(dims * (1 + offsets[0]), log_z, yerr=log_z_err, ls='', capsize=3.0, marker='x', label='Initial')
    axs[i].errorbar(dims * (1 + offsets[1]), final_log_z, yerr=final_log_z_err, ls='', capsize=3.0, marker='.', label='Resampled')
    axs[i].grid(axis='y')
    axs[i].set_xlabel('Dimensions')
    axs[i].set_xscale("log", base=2)
    axs[i].set_xticks(dims)
    

axs[0].set_title('Gaussian')
axs[1].set_title('Gaussian Mixture')
axs[0].set_ylabel(r'$\ln (\hat{Z} / Z_{\textrm{True}})$')
axs[1].tick_params(labelright=True)
  
    
# h, l = fig.axes[0].get_legend_handles_labels()
# fig.legend(h, l, frameon=False, ncol=2, loc='lower center', bbox_to_anchor=(0.6, -0.05))
fig.savefig('figures_rerun/resampling.pdf')

## Comparison to nessai

In [ ]:
gaussian_baseline_path = './outdir/baseline_gaussian_*d/'
baseline_gaussian_results = load_all_results(
    gaussian_baseline_path, file='summary.json'
)

In [ ]:
gmm_baseline_path = './outdir/baseline_gmm_paper_*'
baseline_gmm_results = load_all_results(
    gmm_baseline_path, file='summary.json'
)


In [ ]:
rosenbrock_path = './outdir_rerun/ins_rosenbrock_*d/'
rosenbrock_results = load_all_results(rosenbrock_path, file="summary.json")

In [ ]:
rosenbrock_baseline_path = './outdir/baseline_rosenbrock_*d/'
baseline_rosenbrock_results = load_all_results(rosenbrock_baseline_path, file="summary.json")

In [ ]:
all_results = [gaussian_results, gmm_results, rosenbrock_results]
all_baselines = [baseline_gaussian_results, baseline_gmm_results, baseline_rosenbrock_results]

In [ ]:
references = ["gaussian", "gaussian", "ins"]
shift = 0.06
quantiles = [0.16, 0.84]

n_cols = len(all_results)
n_rows = 5


figsize = (default_figsize[0], 1.8 * default_figsize[1])
hspace = 0.1
# Ratio of height to width scaled by the number of subplots in each dimension
wspace = hspace * (figsize[1] / figsize[0]) * (n_cols / n_rows)

fig, axs = plt.subplots(
    n_rows, n_cols,
    figsize=figsize,
    sharex="col", sharey="row",
    gridspec_kw={"wspace": wspace, "hspace": hspace, "width_ratios": [5, 5, 3]}
)

for (col_idx, results), baseline, ref in zip(enumerate(all_results), all_baselines, references):

    dims = np.array(list(results.keys()))
    print("Found dims")

    def get_mean_std(res, key):
        mu = np.array([d[key].mean() for d in res.values()])
        std = np.array([d[key].std() for d in res.values()])
        # std = np.array([np.quantile(d[key], quantiles) for d in res.values()]).T
        # std[0, :] = np.abs(mu - std[0, :])
        # std[1, :] = np.abs(std[1, :] - mu)
        
        return mu, std

    evidence = get_mean_std(results, 'final_log_evidence')
    baseline_evidence = get_mean_std(baseline, 'log_evidence')

    error = get_mean_std(results, 'final_log_evidence_error')
    baseline_error = get_mean_std(baseline, 'log_evidence_error')

    evals = get_mean_std(results, 'likelihood_evaluations')
    baseline_evals = get_mean_std(baseline, 'likelihood_evaluations')

    times = get_mean_std(results, 'sampling_time')
    baseline_times = get_mean_std(baseline, 'sampling_time')

    ess = get_mean_std(results, 'ess')
    baseline_ess = get_mean_std(baseline, 'ess')

    print('Difference in mean values (ins vs ns)')
    print(f'Evidence mean: {evidence[0]} vs {baseline_evidence[0]}')
    print(f'Evidence std: {evidence[1]} vs {baseline_evidence[1]}')
    print(f'Evals: {evals[0]} vs {baseline_evals[0]}')
    print(f'Ratio: {baseline_evals[0] / evals[0]}')
    print(f'ESS: {ess[0]} vs {baseline_ess[0]}')

    dims_ins = (1 - shift) * dims
    dims_ns = (1 + shift) * dims
    marker = '.'
    ns_marker = 'x'
    
    label = r'$\ln (\hat{Z} / Z_\textrm{Ref})$'
    
    if ref == "gaussian":
        offset = np.array([t for d, t in truth.items() if d in dims])
    elif ref == "ins":
        offset = evidence[0]
    elif ref == "baseline":
        offset = baseline[0]
    else:
        raise ValueError()
    
    axs[0, col_idx].errorbar(dims_ins, evidence[0] - offset, yerr=evidence[1], ls='',
                    marker=marker)
    axs[0, col_idx].errorbar(dims_ns, baseline_evidence[0] - offset, yerr=baseline_evidence[1], 
                    ls='', marker=ns_marker)

    axs[1, col_idx].errorbar(dims_ins, error[0], yerr=error[1], ls='',
                    marker=marker)
    axs[1, col_idx].errorbar(dims_ns, baseline_error[0], yerr=baseline_error[1], ls='',
                    marker=ns_marker)

    print(evals[1])
    
    axs[2, col_idx].errorbar(dims_ins, evals[0], yerr=evals[1], ls='', marker=marker, capsize=3.0)
    axs[2, col_idx].errorbar(dims_ns, baseline_evals[0], yerr=baseline_evals[1], ls='', 
                    marker=ns_marker)
    axs[2, col_idx].set_yscale('log')

    axs[3, col_idx].errorbar(dims_ins, times[0], yerr=times[1], ls='', marker=marker, capsize=3.0)
    axs[3, col_idx].errorbar(dims_ns, baseline_times[0], yerr=baseline_times[1], ls='',
                    marker=ns_marker)
    axs[3, col_idx].set_yscale('log')

    axs[4, col_idx].errorbar(dims_ins, ess[0], yerr=ess[1], ls='', marker=marker, capsize=3.0)
    axs[4, col_idx].errorbar(dims_ns, baseline_ess[0], yerr=baseline_ess[1], ls='',
                    marker=ns_marker)
    
    axs[-1, col_idx].set_xlabel('Dimensions')
    axs[-1, col_idx].set_xscale('log', base=2)
    axs[-1, col_idx].set_xticks(dims.tolist())
    
axs[0, 0].set_ylabel(label)  
axs[1, 0].set_ylabel(r"$\sigma [\ln \hat{Z}]$")    
axs[2, 0].set_ylabel('Likelihood\nevaluations')    
axs[3, 0].set_ylabel('Wall time [s]')
axs[4, 0].set_ylabel('ESS')
    
for a in axs.flatten():
    a.grid(axis='y', which="major")

axs[0, 0].set_title("Gaussian")
axs[0, 1].set_title("Gaussian Mixture")
axs[0, 2].set_title("Rosenbrock")

for i in range(n_rows):
    axs[i, -1].tick_params(labelright=True)

# plt.tight_layout()

fig.savefig("figures_rerun/comparison_all.pdf")



## Error distribution

In [ ]:
fig, axs = plt.subplots(1, 2, sharey=True)
key = "final_log_evidence"
for j, results in enumerate([gaussian_results, gmm_results]):
    N = 50
    x = np.linspace(0, 1, N)
    x_ref = np.linspace(0, 1, 1001)
    confidence_interval = [0.68, 0.95, 0.997]
    for ci in confidence_interval:
        edge_of_bound = (1. - ci) / 2.
        lower = stats.binom.ppf(1 - edge_of_bound, N, x_ref) / N
        upper = stats.binom.ppf(edge_of_bound, N, x_ref) / N
        # The binomial point percent function doesn't always return 0 @ 0,
        # so set those bounds explicitly to be sure
        lower[0] = 0
        upper[0] = 0
        axs[j].fill_between(x_ref, lower, upper, alpha=0.1, color='grey')

    colours = sns.color_palette("mako", n_colors=5)

    for i, (dims, res) in enumerate(results.items()):
        Z_hat = np.exp(res[key])
        std = np.abs(res[key + "_error"] * Z_hat)
        dist = stats.norm(loc=np.exp(truth[dims]), scale=np.mean(std))
        pp = sm.ProbPlot(Z_hat, dist=dist)
        axs[j].plot(x, pp.sample_percentiles, label=dims, c=colours[i])
#     axs[j].legend(title="Dimensions")
    axs[j].set_xlim([0, 1])
    axs[j].set_ylim([0, 1])
    axs[j].set(adjustable='box', aspect='equal')
axs[0].set_title("Gaussian")
axs[1].set_title("Gaussian Mixture")

axs[0].set_ylabel("Empirical CDF")
axs[0].set_xlabel("Theoretical CDF")
axs[1].set_xlabel("Theoretical CDF")

handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, title="Dimensions", ncols=5, loc="lower center", bbox_to_anchor=(0.5, -0.05))
fig.savefig("figures_rerun/uncertainty_pp_plots.pdf")
plt.show()

## Rosenbrock p-values

In [ ]:
frames = []
for k, v in baseline_rosenbrock_results.items():
    df_tmp = pd.DataFrame({
        "dims": k * np.ones(len(v)),
        "p_value": v["p_value"]
    })
    frames.append(df_tmp)
df = pd.concat(frames)

In [ ]:
with sns.plotting_context(
    rc={
        "xtick.top": False,
    }
), sns.axes_style("ticks"):
    g = sns.FacetGrid(df, row="dims", aspect=3, height=1)
    bins = np.linspace(0, 1.0, 8)
    g.map_dataframe(
        sns.histplot, x="p_value", fill=True, alpha=1, element="step", clip_on=False,
        bins=bins, hue="dims", hue_norm=(0, 10), palette="RdYlBu",
#         histtype="step", range=[0, 1],
#         hist_kws={"range": [0,1]}
    )
#     g.map(plt.axhline, y=0, lw=2, clip_on=False)
# g.fig.subplots_adjust(hspace=-.5)
axes = g.axes.flatten()

for dims, ax in zip([2, 4, 8], axes):
    ax.text(0.65, .6, f"{dims}-dimensional", fontweight="bold", color=None, ha="left", va="center", transform=ax.transAxes)
    ax.set_ylabel("")

axes[-1].set_xlabel("$p$-value")
    
# g.map(label, "dims")

g.set_titles("")
g.set(yticks=[])
g.despine(left=True, bottom=False)
g.tight_layout()

g.fig.subplots_adjust(hspace=0.1)
g.savefig("figures/rosenbrock_p_values.pdf")